# パッケージインストール

In [1]:
!pip install onnxruntime 
!pip install cryptography

     |████████████████████████████████| 4.9 MB 27.9 MB/s 
     |████████████████████████████████| 3.6 MB 21.8 MB/s 


# サンプルモデルダウンロード

In [2]:
!wget https://github.com/onnx/models/blob/main/vision/classification/resnet/model/resnet50-v1-12.onnx?raw=true -O resnet50-v1-12.onnx

--2022-03-16 05:27:48--  https://github.com/onnx/models/blob/main/vision/classification/resnet/model/resnet50-v1-12.onnx?raw=true
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/onnx/models/raw/main/vision/classification/resnet/model/resnet50-v1-12.onnx [following]
--2022-03-16 05:27:48--  https://github.com/onnx/models/raw/main/vision/classification/resnet/model/resnet50-v1-12.onnx
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/onnx/models/main/vision/classification/resnet/model/resnet50-v1-12.onnx [following]
--2022-03-16 05:27:48--  https://raw.githubusercontent.com/onnx/models/main/vision/classification/resnet/model/resnet50-v1-12.onnx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.19

# ONNXモデル読み込み確認

In [3]:
import cv2 as cv
import onnxruntime

model_path = 'resnet50-v1-12.onnx'

onnx_session = onnxruntime.InferenceSession(
    model_path,
    providers=['CPUExecutionProvider'],
)

print(onnx_session.get_inputs()[0])
print(onnx_session.get_outputs()[0])

NodeArg(name='data', type='tensor(float)', shape=[1, 3, 224, 224])
NodeArg(name='resnetv17_dense0_fwd', type='tensor(float)', shape=[1, 1000])


# キー生成方式での暗号化・複合化

In [4]:
from cryptography.fernet import Fernet
from cryptography.fernet import InvalidToken

# キー生成
key = Fernet.generate_key()

# キー保存
with open('./key.txt', 'wb') as f:
    f.write(key)
print(key)

b'PZz9_vQzHQtJvfTFv_F5iTIiWs7oZwSPs8A6rtapC0s='


In [5]:
# ONNXモデルをバイナリモードでロード
onnx_data = None
with open('resnet50-v1-12.onnx', 'rb') as onnx_file:
    onnx_data = onnx_file.read()

# Fernetオブジェクトを指定キーで生成
fernet = Fernet(key)

# 暗号化
encrypt_data = fernet.encrypt(onnx_data)

# 暗号化したデータをファイルに書き込む
with open('resnet50-v1-12.dat', 'wb') as f:
    f.write(encrypt_data)

In [6]:
# キーの読み込み
with open('key.txt', 'rb') as f:
    key = f.read()
print(key)

# 暗号化したデータの読み込み
with open('resnet50-v1-12.dat', 'rb') as f:
    encryp_data = f.read()

# Fernetオブジェクトを指定キーで生成
fernet = Fernet(key)

# 復号化
try:
    decrypt_data = fernet.decrypt(encryp_data)
except InvalidToken:
    print('InvalidToken')

b'PZz9_vQzHQtJvfTFv_F5iTIiWs7oZwSPs8A6rtapC0s='


In [7]:
# モデルロード
import cv2 as cv
import onnxruntime

# 複合化したONNXモデルのロード
onnx_session = onnxruntime.InferenceSession(
    decrypt_data,
    providers=['CPUExecutionProvider'],
)

print(onnx_session.get_inputs()[0])
print(onnx_session.get_outputs()[0])

NodeArg(name='data', type='tensor(float)', shape=[1, 3, 224, 224])
NodeArg(name='resnetv17_dense0_fwd', type='tensor(float)', shape=[1, 1000])


# パスワード指定方式での暗号化・複合化

In [8]:
import base64
from cryptography.fernet import Fernet
from cryptography.fernet import InvalidToken
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC

# ONNXモデルをバイナリモードでロード
with open('resnet50-v1-12.onnx', 'rb') as onnx_file:
    onnx_data = onnx_file.read()

# パスワード
password_text = 'P@ssw0rd'

# ソルト付与・ストレッチング実施用インスタンス生成
# ソルトはユーザー毎にランダム値を生成すべきだが、サンプルではひとまず固定値を使用
salt = b'salt'  
stretching_iterations = 390000
kdf = PBKDF2HMAC(
    algorithm=hashes.SHA256(),
    length=32,
    salt=salt,
    iterations=stretching_iterations,
)

# キー生成
password = password_text.encode("utf-8")
key = base64.urlsafe_b64encode(kdf.derive(password))

# Fernetオブジェクトを指定キーで生成
fernet = Fernet(key)

# 暗号化
encrypt_data = fernet.encrypt(onnx_data)

# 暗号化したデータをファイルに書き込む
with open('resnet50-v1-12-password.dat', 'wb') as f:
    f.write(encrypt_data)

In [9]:
# 暗号化したデータの読み込み
with open('resnet50-v1-12-password.dat', 'rb') as f:
    encryp_data = f.read()

# パスワード
password_text = 'P@ssw0rd'

# ソルト付与・ストレッチング実施用インスタンス生成
# ソルトはユーザー毎にランダム値を生成すべきだが、サンプルではひとまず固定値を使用
salt = b'salt'  
stretching_iterations = 390000
kdf = PBKDF2HMAC(
    algorithm=hashes.SHA256(),
    length=32,
    salt=salt,
    iterations=stretching_iterations,
)

# キー生成
password = password_text.encode("utf-8")
key = base64.urlsafe_b64encode(kdf.derive(password))

# Fernetオブジェクトを指定キーで生成
fernet = Fernet(key)

# 復号化
try:
    decrypt_data = fernet.decrypt(encryp_data)
except InvalidToken:
    print('InvalidToken')

In [10]:
# モデルロード
import cv2 as cv
import onnxruntime

# ONNXランタイム
onnx_session = onnxruntime.InferenceSession(
    decrypt_data,
    providers=['CPUExecutionProvider'],
)

print(onnx_session.get_inputs()[0])
print(onnx_session.get_outputs()[0])

NodeArg(name='data', type='tensor(float)', shape=[1, 3, 224, 224])
NodeArg(name='resnetv17_dense0_fwd', type='tensor(float)', shape=[1, 1000])
